In [1]:
import pandas as pd
from collections import Counter
import ast

In [2]:
df = pd.read_csv('purposes_added_metadata_v2.csv')

In [3]:
df_pred = pd.read_csv('Trained_predicted_purposes.csv')

In [5]:
df_pred.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,pmc_json_files,url,s2_id,referenced_by_count,JournalName_DOI,tags,TagCount,multihot_labels,predicted_labels,predicted_count
0,4i02f1ji,a08d4b749b4d5b73acfa15344195a097a9699c96,MedRxiv,Clinical course and risk factors for mortality...,10.1101/2020.04.24.20072611,NaN,NaN,medrxiv,Background: Patients with pre-existing cirrhos...,2020-04-28,...,NaN,https://doi.org/10.1101/2020.04.24.20072611,216653286.0,0,NaN,[],0,NaN,['Quantitative Data and Analysis'],1
1,3q6idrmz,b2aef9bcd1a7b70237ed02ef4004c94f86017151,Elsevier; Medline; PMC,The AAD COVID-19 Registry: Crowdsourcing Derma...,10.1016/j.jaad.2020.04.045,PMC7162762,32305438.0,els-covid,The AAD COVID-19 Registry: Crowdsourcing Derma...,2020-04-17,...,NaN,https://www.sciencedirect.com/science/article/...,215790286.0,54,Journal of the American Academy of Dermatology,[],0,NaN,['Quantitative Data and Analysis'],1
2,ys7z7j8j,ff2e6962ae80b7f17be32dd59513bdeb3a2d6e92,Elsevier; Medline; PMC,Keratoconjunctivitis as the initial medical pr...,10.1016/j.jcjo.2020.03.003,PMC7124283,32284146.0,els-covid,Abstract We present a case of coronavirus dise...,2020-04-02,...,document_parses/pmc_json/PMC7124283.xml.json,https://api.elsevier.com/content/article/pii/S...,214758418.0,219,Canadian Journal of Ophthalmology,[],0,NaN,[],0
3,tgpqnoq2,129a24f0574121f32e5bb5bb3495d4ab684e79b6,MedRxiv,Heparin-induced thrombocytopenia is associated...,10.1101/2020.04.23.20076851,NaN,NaN,medrxiv,Background Coronavirus infectious disease 2019...,2020-04-28,...,NaN,http://medrxiv.org/cgi/content/short/2020.04.2...,216588708.0,42,NaN,[],0,NaN,['Quantitative Data and Analysis'],1
4,bvdyhq6v,e1000a407a8d3581502550fff91c69ffed0f760e,Elsevier; Medline; PMC,Echocardiography in the Time of COVID-19,10.1016/j.echo.2020.04.011,PMC7146691,32503705.0,els-covid,Echocardiography in the Time of COVID-19,2020-04-10,...,NaN,https://www.sciencedirect.com/science/article/...,215550941.0,33,Journal of the American Society of Echocardiog...,[],0,NaN,"['Theoretical Frameworks or Models', 'Comparat...",2


In [6]:
frequency = df['TagCount'].value_counts()
frequency

TagCount
1    4133
0    2433
2    1946
3     714
4      84
5       6
7       1
6       1
Name: count, dtype: int64

In [7]:
df['tags'] = df['tags'].apply(ast.literal_eval)

In [8]:
all_tags = [tag for tags_list in df['tags'] for tag in tags_list]

tag_frequency = Counter(all_tags)

In [9]:
tag_frequency_df = pd.DataFrame(tag_frequency.items(), columns=['Tag', 'Frequency']).sort_values(by='Frequency', ascending=False)
tag_frequency_df

,Tag,Frequency
3,Quantitative Data and Analysis,3761
0,Methodologies or Experimental Designs,2268
1,Theoretical Frameworks or Models,1880
5,Comparative Works,1066
2,Research Gaps,945
4,Literature Review,487
6,Exploring Interdisciplinary Connection,139


In [11]:
frequency = df_pred['predicted_count'].value_counts()
frequency

predicted_count
1    1484
0     538
2     365
3      46
Name: count, dtype: int64

In [10]:
df_pred['predicted_labels'] = df_pred['predicted_labels'].apply(ast.literal_eval) 

In [11]:
df.set_index('cord_uid', inplace=True)
df_pred.set_index('cord_uid', inplace=True)

In [12]:
df.loc[df_pred.index, 'tags'] = df_pred['predicted_labels']

In [10]:
df['TagCount'] = df['tags'].apply(len)

In [13]:
frequency = df['TagCount'].value_counts()
frequency

TagCount
1    4133
0    2433
2    1946
3     714
4      84
5       6
7       1
6       1
Name: count, dtype: int64

In [14]:
all_tags = [tag for tags_list in df['tags'] for tag in tags_list]

tag_frequency = Counter(all_tags)

In [15]:
tag_frequency_df_updated = pd.DataFrame(tag_frequency.items(), columns=['Tag', 'Frequency']).sort_values(by='Frequency', ascending=False)
tag_frequency_df_updated

,Tag,Frequency
3,Quantitative Data and Analysis,4624
0,Methodologies or Experimental Designs,2968
1,Theoretical Frameworks or Models,2183
5,Comparative Works,1223
2,Research Gaps,1124
4,Literature Review,624
6,Exploring Interdisciplinary Connection,152


In [23]:
def replace_empty_tags(tags):
    return tags if isinstance(tags, list) and len(tags) > 0 else ['miscellaneous'] #replace any empty lists with 'miscelleanous' tag

In [24]:
df['tags'] = df['tags'].apply(replace_empty_tags)

In [25]:
all_tags = [tag for tags_list in df['tags'] for tag in tags_list]

tag_frequency = Counter(all_tags)

In [26]:
tag_frequency_df_updated_v2 = pd.DataFrame(tag_frequency.items(), columns=['Tag', 'Frequency']).sort_values(by='Frequency', ascending=False)
tag_frequency_df_updated_v2

,Tag,Frequency
3,Quantitative Data and Analysis,4624
0,Methodologies or Experimental Designs,2968
1,Theoretical Frameworks or Models,2183
5,Comparative Works,1223
2,Research Gaps,1124
4,Literature Review,624
7,miscellaneous,538
6,Exploring Interdisciplinary Connection,152


In [28]:
df.reset_index(inplace=True)

In [29]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,referenced_by_count,JournalName_DOI,tags,TagCount
0,iua8c4hy,300ff0def92740f7e1142036f9e6ae30845e13d2,MedRxiv,COVID-19 Outbreak in Oman: Model-Driven Impact...,10.1101/2020.04.02.20050666,NaN,NaN,medrxiv,Motivated by the rapid spread of COVID-19 all ...,2020-04-06,...,NaN,NaN,document_parses/pdf_json/300ff0def92740f7e1142...,NaN,https://doi.org/10.1101/2020.04.02.20050666,215782459.0,10,NaN,"[Methodologies or Experimental Designs, Theore...",4
1,xph08fwv,9a3e8c974bc68c29b0273ee771553046d7f37930,MedRxiv,Automated and semi-automated contact tracing: ...,10.1101/2020.04.14.20063636,NaN,NaN,medrxiv,Introduction Traditional approaches to case-fi...,2020-04-17,...,NaN,NaN,document_parses/pdf_json/9a3e8c974bc68c29b0273...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216055053.0,2,NaN,"[Literature Review, Methodologies or Experimen...",4
2,aosmo568,ca6949e3e039a0e4098644b0b2df90606ae3a7ee,MedRxiv,Efficacy of remdesivir versus placebo for the ...,10.1101/2020.04.09.20059196,NaN,NaN,medrxiv,Background: In spite of the global containment...,2020-04-14,...,NaN,NaN,document_parses/pdf_json/ca6949e3e039a0e409864...,NaN,http://medrxiv.org/cgi/content/short/2020.04.0...,215782112.0,1,NaN,"[Literature Review, Methodologies or Experimen...",4
3,ujomta30,d831dbf38025a44e0436fd11c52af4db9eb7c5b0,MedRxiv,Hydroxychloroquine Versus COVID-19: A Rapid Sy...,10.1101/2020.04.14.20065276,NaN,NaN,medrxiv,Background: Coronavirus Disease 2019 (COVID-19...,2020-04-20,...,NaN,NaN,document_parses/pdf_json/d831dbf38025a44e0436f...,NaN,http://medrxiv.org/cgi/content/short/2020.04.1...,216035656.0,15,NaN,"[Literature Review, Methodologies or Experimen...",4
4,xwlzq3m3,44eec5c0f84d7069033431dcb5a6adeb01b8ee0c,Elsevier; Medline; PMC,Clinical characteristics and diagnostic challe...,10.1016/j.jfma.2020.04.007,PMC7161491,32307322.0,els-covid,Background/Purpose Current studies on pediatri...,2020-04-16,...,NaN,NaN,document_parses/pdf_json/44eec5c0f84d706903343...,document_parses/pmc_json/PMC7161491.xml.json,https://www.ncbi.nlm.nih.gov/pubmed/32307322/;...,215787843.0,157,Journal of the Formosan Medical Association,"[Literature Review, Methodologies or Experimen...",4


In [30]:
df.to_csv("purposes_metadata_final.csv", index=False)